-----------------------
## 1) New way of scraping  11.4.2024
-----------------------

In [1]:
from datetime import datetime
import pandas as pd
from tqdm import tqdm  
import requests 

import os
os.chdir("c:\\Users\\jirka\\Documents\\MyProjects\\Sreality")

from run import Runner
runner = Runner()

## Testing requests

In [ ]:
# Just testing count
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/count"
    result =  session.request(method="GET", url=url, headers=headers).json()
    print(result)
    url_filter = f"https://www.sreality.cz/api/cs/v2/estates?category_main_cb=1&category_type_cb=1&per_page=1&page=1"
    result2 =  session.request(method="GET", url=url_filter).json()
    print(result2)

In [ ]:
headers = {"User-Agent": "Mozilla/5.0"}

with requests.Session() as session:
    url = "https://www.sreality.cz/api/cs/v2/estates/95413580"
    result =  session.request(method="GET", url=url, headers=headers).json()
    display(result)


## Full Run with follow-up scraping and GeoData update

In [ ]:
res = runner.scrape_and_update_run(scrape_prodej_byty=True, scrape_all=True)

## Inserting DFs to DB

In [ ]:
runner = Runner()
new_estates = runner.input_all_estates_to_db()
print(new_estates.columns)
#new_estates.describe().T

In [2]:
runner = Runner()
new_prices = runner.input_all_prices_to_db()
print(new_prices.columns)
#new_prices.describe().T

100%|██████████| 129/129 [12:00<00:00,  5.58s/it]


There was an error in file to_fix
4623912


,estate_id,price,crawled_at,code;name;category_type_cb;category_main_cb;category_sub_cb;rooms;size_meters;locality;latitude;longitude;price;timestamp,code;name;price;timestamp,code;price;timestamp
0,NaN,NaN,NaN,926332236;Prodej bytu 2+kk 56Â mÂ˛;1;1;4;2+kk;...,NaN,NaN
1,NaN,NaN,NaN,1346397260;Prodej bytu 3+kk 70Â mÂ˛;1;1;6;3+kk...,NaN,NaN
2,NaN,NaN,NaN,1748108620;Prodej bytu 3+kk 64Â mÂ˛;1;1;6;3+kk...,NaN,NaN
3,NaN,NaN,NaN,2776523852;Prodej bytu 3+kk 72Â mÂ˛;1;1;6;3+kk...,NaN,NaN
4,NaN,NaN,NaN,3868194124;Prodej bytu 2+kk 67Â mÂ˛;1;1;4;2+kk...,NaN,NaN


Index(['estate_id', 'price', 'crawled_at',
       'code;name;category_type_cb;category_main_cb;category_sub_cb;rooms;size_meters;locality;latitude;longitude;price;timestamp',
       'code;name;price;timestamp', 'code;price;timestamp'],
      dtype='object')


In [ ]:
print(new_prices)

In [ ]:
new_estates["start_of_offer"].value_counts() 

In [ ]:
new_estates.tail(2)

## Test Utils

In [ ]:
from run import Runner
runner = Runner()

In [ ]:
df_new = runner.utils.prep_df_new_estates("20240422_1647.json")
print(len(df_new.columns))
print(df_new.columns)
df_new.tail(2)

In [ ]:
x = runner.utils.assign_location_to_df(df_new)
display(x.head(2))
print(len(x.columns))
print(x.columns)

In [ ]:
timestamp, _ = runner.utils.generate_timestamp()
print(timestamp)
#data_manager._insert_new_estate(df_new, timestamp)

## Prepare Geo - JSONs

In [ ]:
source_folder= "c:\\Users\\jirka\\Documents\\MyProjects\\Sreality\\data_2024\\estate_details"
files = os.listdir(source_folder)
file_info = []

for file_name in files:
        file_path = os.path.join(source_folder, file_name)
        if os.path.isfile(file_path):
            size = os.path.getsize(file_path)
            file_info.append((file_name, size))

# Sort files by size
sorted_files = sorted(file_info, key=lambda x: x[1])

sorted_names = []
for file_name, size in sorted_files:
    sorted_names.append(file_name)
    print(f"{file_name}: {size} bytes")

In [ ]:
runner.geodata.enrich_jsons_with_geodata(list_of_jsons=None)

## Table managment test

In [ ]:
d = runner.data_manager.get_all_rows("estate_detail")
print(len(d))
d_codes = d["code"].unique()
print(len(d_codes))
print(d_codes[0])
print(type(d_codes[0]))

display(d.head(2))
print(len(d.columns))
#display(sorted(d.columns))


In [ ]:
runner.data_manager.clear_table("price_history")

In [ ]:
runner.data_manager.drop_table("estate_detail")

In [ ]:
runner.data_manager.create_table("price_history")

In [ ]:
upload = pd.DataFrame(data = [[42, "Prodej luxsního bejváku", "Byt 14+kk daleko od centra, ale v klidu.",
                               1, 6, 1, 111, "M&M reality", 2, 49.827491575, 18.1654443,
                               0, 0, 111083, 65, 14829, 12, 10, 4730, 112,
                               0, 0, 0, 0, 0, 2, 0, 79, "Družstevní", 0, 1, 1, 0,
                               3, 0, 870824, 1, 2,
                               ### Not mandatory:
                               "Cena je finální", 123,
                               "Dnes", "cihla", "Novostavba", "Družstevní", "2.patro",
                               42, 43, 2, "true"
                               ]],
                      columns=['code', 'description', 'meta_description', 
                               'category_main_cb', 'category_type_cb', 'category_sub_cb', 'broker_id', 'broker_company',
                            'furnished', 'latitude', 'longitude', 'locality', 'city', 'district',
                            'region', 'object_type', 'parking_lots', 'locality_street_id',
                            'locality_district_id', 'locality_ward_id', 'locality_region_id',
                            'locality_quarter_id', 'locality_municipality_id',
                            'locality_country_id', 'terrace', 'balcony', 'loggia', 'basin',
                            'cellar', 'building_type', 'object_kind', 'ownership', 'low_energy',
                            'easy_access', 'building_condition', 'garage', 'room_count_cb', 'energy_efficiency_rating_cb', 
                            ### Not mandatory:
                            'note_about_price', 'id_of_order',
                            'last_update', 'material', 'age_of_building', 'ownership_type', 'floor',
                            'usable_area', 'floor_area', 'energy_efficiency_rating', "no_barriers"
                            ])

runner.data_manager._insert_new_estate(upload, "2024-04-21 23:22:44")

#TODO: how about unit test for this?
#? should fail predictably as it breakes foreign key constraints i made:)
upload = pd.DataFrame(data = [["911", "4242", "12345000"]],
                      columns=["estate_id", "batch_id", "price"])

runner.data_manager.insert_new_price(upload)

In [ ]:
describe = d["id"].describe()
describe

In [ ]:
print(describe["min"])

print(d["id"].min())
      

## Konec